In [1]:
import math
import re
import random
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

import requests
import xml.etree.ElementTree as ET
import xml.dom.minidom
from bs4 import BeautifulSoup
from selenium import webdriver

from IPython.display import Image
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', None)

from pykrx import stock
import FinanceDataReader as fdr
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import FinanceDataReader as fdr
import finterstellar as fs


import talib.abstract as ta
from talib import MA_Type

import statistics as ss

import statsmodels.api as sm # 원래 여깄었는데
from statsmodels.regression.linear_model import OLS # 여기에도 추가됨
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

import scipy as sp
from scipy import stats
from scipy.stats import skew, kurtosis
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree, fcluster

# from imblearn.under_sampling import TomekLinks

# from sklearn.datasets import load_iris, load_wine, load_diabetes#, load_boston
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, Binarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.linear_model import LogisticRegression # 이름은 회귀인데 시그모이드함수를 이용한 분류모델임 분류분류분류
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, RandomForestRegressor
# from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVR, SVC
from sklearn.cluster import KMeans, DBSCAN#, hierarchical
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import precision_score, precision_recall_curve#, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve#, plot_roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import silhouette_samples, silhouette_score

from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
# from catboost import CatBoostRegressor, CatBoostClassifier     # pip install catboost

import optuna

import tensorflow as tf
import keras

from keras import backend as K

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator

from keras import activations as act
from keras import optimizers as opt
from keras import losses as loss
from keras import metrics

from keras.initializers import GlorotNormal  #Xavier

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
keras.utils.set_random_seed(SEED)

import warnings
warnings.filterwarnings(action='ignore')

import bt

from sqlalchemy import create_engine, MetaData, text, Float, Table, Column, Integer, String
from sqlalchemy.engine import reflection
import cx_Oracle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#-------------------- 차트 관련 속성 (plotly 화 - 분석시에만 키기) -----------
# pd.options.plotting.backend = "plotly"

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

pd.set_option('max_colwidth', None)
plt.rcParams['font.family']

findfont: Font family ['NanumBarunGothic'] not found. Falling back to DejaVu Sans.


['Malgun Gothic']

In [2]:
SPY_PER = pd.read_excel("./datasets/SPY_PER.xlsx")[4:]
SPY_PER.columns=["Date","SPY_PER","YOY"]
SPY_PER["Date"] = SPY_PER["Date"].astype("datetime64")
SPY_PER=SPY_PER.set_index("Date")
SPY_PER=SPY_PER.drop("YOY",axis=1)
SPY_PER

,SPY_PER
Date,
2023-06-13,24.72
2023-06-12,24.72
2023-06-11,24.49
2023-06-09,24.49
2023-06-08,24.46
...,...
1871-05-01,12.15
1871-04-01,11.85
1871-03-01,11.53


In [3]:
SPY_EPS = pd.read_excel("./datasets/SPY_EPS.xlsx")[4:]
SPY_EPS.columns=["Date","SPY_EPS","YOY"]
SPY_EPS["Date"] = SPY_EPS["Date"].astype("datetime64")
SPY_EPS=SPY_EPS.set_index("Date")
SPY_EPS=SPY_EPS.drop("YOY",axis=1)
SPY_EPS

,SPY_EPS
Date,
2023-03-01,175.52
2023-02-01,174.6
2023-01-01,173.67
2022-12-01,172.75
2022-11-01,177.58
...,...
1871-05-01,0.4
1871-04-01,0.4
1871-03-01,0.4


In [10]:
SPY_df = pd.concat([SPY_PER,SPY_EPS],axis=1)
SPY_df = SPY_df.sort_index()
SPY_df=SPY_df.dropna()
SPY_df

,SPY_PER,SPY_EPS
Date,,
1871-01-01,11.1,0.4
1871-02-01,11.25,0.4
1871-03-01,11.53,0.4
1871-04-01,11.85,0.4
1871-05-01,12.15,0.4
...,...,...
2022-10-01,20.43,182.4
2022-11-01,21.14,177.58
2022-12-01,22.96,172.75


In [11]:
SPY_df2 = SPY_df.diff(12)
SPY_df2=SPY_df2.dropna()
SPY_df2

,SPY_PER,SPY_EPS
Date,,
1872-01-01,1.05,0.0
1872-02-01,0.65,0.01
1872-03-01,0.76,0.01
1872-04-01,0.78,0.01
1872-05-01,0.48,0.01
...,...,...
2022-10-01,-3.96,-0.51
2022-11-01,-3.37,-12.81
2022-12-01,-0.67,-25.12


In [12]:
SPY_signal_list = []
for i in range(len(SPY_df2)):
    if (SPY_df2["SPY_PER"][i]<0)&(SPY_df2["SPY_EPS"][i]<0):
        SPY_signal_list.append("하락")

    elif (SPY_df2["SPY_PER"][i]<0)&(SPY_df2["SPY_EPS"][i]>=0):
        SPY_signal_list.append("회복")

    elif (SPY_df2["SPY_PER"][i]>=0)&(SPY_df2["SPY_EPS"][i]<0):
        SPY_signal_list.append("정상")

    else:
        SPY_signal_list.append("상승")

SPY_df2['SPY_signal'] = SPY_signal_list

In [13]:
SPY_df2

,SPY_PER,SPY_EPS,SPY_signal
Date,,,
1872-01-01,1.05,0.0,상승
1872-02-01,0.65,0.01,상승
1872-03-01,0.76,0.01,상승
1872-04-01,0.78,0.01,상승
1872-05-01,0.48,0.01,상승
...,...,...,...
2022-10-01,-3.96,-0.51,하락
2022-11-01,-3.37,-12.81,하락
2022-12-01,-0.67,-25.12,하락


In [14]:
signal_list = [SPY_df2['SPY_signal'][0]]
idx_list = [SPY_df2['SPY_signal'].index[0]]

for idx, v in SPY_df2.iterrows():
    if v['SPY_signal'] == signal_list[-1]:
        pass
    else :
        signal_list.append(v['SPY_signal'])
        idx_list.append(idx)



In [15]:
cycle_df = pd.DataFrame([signal_list , idx_list] ,index=['signal','date']).T
cycle_df

,signal,date
0,상승,1872-01-01
1,회복,1872-09-01
2,상승,1872-10-01
3,회복,1872-11-01
4,상승,1873-02-01
...,...,...
211,정상,2020-01-01
212,상승,2021-03-01
213,회복,2021-06-01
214,하락,2022-10-01


In [17]:
cycle_df.to_csv("EPS%PER경기국면분할2.csv")